<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [2]:
#loading the data
house = pd.read_csv('./housing.csv')

In [4]:
#cleaning
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
house = house.drop(['MiscFeature', 'Fence', 'PoolQC', 'FireplaceQu', 'Alley', 'LotFrontage'], axis = 1)

In [7]:
#confirm that the above columns have been dropped
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 non-

In [8]:
#fill missing object types with 'NA'
house['MasVnrType'].fillna(value='NA', inplace=True)
house['BsmtQual'].fillna(value='NA', inplace=True)
house['BsmtCond'].fillna(value='NA', inplace=True)
house['BsmtExposure'].fillna(value='NA', inplace=True)
house['BsmtFinType1'].fillna(value='NA', inplace=True)
house['BsmtFinType2'].fillna(value='NA', inplace=True)
house['Electrical'].fillna(value='NA', inplace=True)
house['GarageType'].fillna(value='NA', inplace=True)
house['GarageYrBlt'].fillna(value='NA', inplace=True)
house['GarageFinish'].fillna(value='NA', inplace=True)
house['GarageQual'].fillna(value='NA', inplace=True)
house['GarageCond'].fillna(value='NA', inplace=True)

#fill missing float64 types with '0.0'
house['MasVnrArea'].fillna(value=0.0, inplace=True)

In [9]:
#confirm that all the missing values have been filled
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 non-

In [10]:
# remove any houses that are not residential from the dataset.
#     A --> Agriculture
#     C --> Commercial
#     I --> Industrial

house['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [11]:
#Removing 'C (all)'

house = house[~(house['MSZoning'] == 'C (all)')]

In [12]:
#Confirming 'C (all)' was removed

house['MSZoning'].value_counts()

RL    1151
RM     218
FV      65
RH      16
Name: MSZoning, dtype: int64

In [13]:
# # Identify fixed features that can predict price.

# # fixed features

# MSSubClass
# MSZoning
# LotFrontage (dropped)
# LotArea
# Street
# Alley (dropped)
# LotShape
# LandContour
# Utilities
# LotConfig
# LandSlope
# Neighborhood
# Condition1
# Condition2
# BldgType
# HouseStyle
# YearBuilt
# YearRemodAdd
# RoofStyle
# Foundation
# BsmtQual
# BsmtExposure
# BsmtFinSF1
# BsmtFinSF2
# BsmtUnfSF
# TotalBsmtSF
# CentralAir
# 1stFlrSF
# 2ndFlrSF
# GrLivArea
# BsmtFullBath
# BsmtHalfBath
# FullBath
# HalfBath
# Bedroom
# Kitchen
# TotRmsAbvGrd
# Fireplaces (dropped)
# GarageType
# GarageYrBlt
# GarageFinish
# GarageCars
# GarageArea
# PavedDrive
# PoolArea
# MoSold
# YrSold
# SaleType


In [14]:
# # renovate-able features:

# OverallQual
# OverallCond
# RoofMatl
# Exterior1st
# Exterior2nd
# MasVnrType
# MasVnrArea
# ExterQual
# ExterCond
# BsmtCond
# BsmtFinType1
# BsmtFinType2
# Heating
# HeatingQC
# Electrical
# LowQualFinSF
# KitchenQual
# Functional
# FireplaceQu
# GarageQual
# GarageCond
# WoodDeckSF
# OpenPorchSF
# EnclosedPorch
# 3SsnPorch
# ScreenPorch
# PoolQC (dropped)
# Fence (dropped)
# MiscFeature (dropped)
# MiscVal
# SaleCondition


In [15]:
# mask df for fixed features

ff_house = house.loc[:, ['MSSubClass',
                         'MSZoning', 
                         'LotArea', 
                         'Street', 
                         'LotShape', 
                         'LandContour', 
                         'LotConfig',
                         'LandSlope',
                         'Neighborhood',
                         'Condition1',
                         'Condition2',
                         'BldgType',
                         'HouseStyle',
                         'YearBuilt',
                         'YearRemodAdd',
                         'RoofStyle',
                         'Foundation',
                         'BsmtQual',
                         'BsmtExposure',
                         'BsmtFinSF1',
                         'BsmtFinSF2',
                         'BsmtUnfSF',
                         'TotalBsmtSF',
                         'CentralAir',
                         '1stFlrSF',
                         '2ndFlrSF',
                         'GrLivArea',
                         'BsmtFullBath',
                         'BsmtHalfBath',
                         'FullBath',
                         'HalfBath',
                         'Bedroom',
                         'Kitchen',
                         'TotRmsAbvGrd',
                        # 'Fireplaces (dropped)',
                         'GarageType',
                         'GarageYrBlt',
                         'GarageFinish',
                         'GarageCars',
                         'GarageArea',
                         'PavedDrive',
                         'PoolArea',
                         'MoSold',
                         'YrSold',
                         'SaleType',
                        ]
                    ]


In [16]:
ff_house

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,PavedDrive,PoolArea,MoSold,YrSold,SaleType
0,60,RL,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,...,Attchd,2003,RFn,2,548,Y,0,2,2008,WD
1,20,RL,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,...,Attchd,1976,RFn,2,460,Y,0,5,2007,WD
2,60,RL,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,...,Attchd,2001,RFn,2,608,Y,0,9,2008,WD
3,70,RL,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,...,Detchd,1998,Unf,3,642,Y,0,2,2006,WD
4,60,RL,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,...,Attchd,2000,RFn,3,836,Y,0,12,2008,WD
5,50,RL,14115,Pave,IR1,Lvl,Inside,Gtl,Mitchel,Norm,...,Attchd,1993,Unf,2,480,Y,0,10,2009,WD
6,20,RL,10084,Pave,Reg,Lvl,Inside,Gtl,Somerst,Norm,...,Attchd,2004,RFn,2,636,Y,0,8,2007,WD
7,60,RL,10382,Pave,IR1,Lvl,Corner,Gtl,NWAmes,PosN,...,Attchd,1973,RFn,2,484,Y,0,11,2009,WD
8,50,RM,6120,Pave,Reg,Lvl,Inside,Gtl,OldTown,Artery,...,Detchd,1931,Unf,2,468,Y,0,4,2008,WD
9,190,RL,7420,Pave,Reg,Lvl,Corner,Gtl,BrkSide,Artery,...,Attchd,1939,RFn,1,205,Y,0,1,2008,WD


In [17]:
MSZoning_dummies = pd.get_dummies(ff_house.MSZoning, prefix='MSZoning')
Street_dummies = pd.get_dummies(ff_house.Street, prefix='Street')
LotShape_dummies = pd.get_dummies(ff_house.LotShape, prefix='LotShape')
LandContour_dummies = pd.get_dummies(ff_house.LandContour, prefix='LandContour')
LotConfig_dummies = pd.get_dummies(ff_house.LotConfig, prefix='LotConfig')
LandSlope_dummies = pd.get_dummies(ff_house.LandSlope, prefix='LandSlope')
Neighborhood_dummies = pd.get_dummies(ff_house.Neighborhood, prefix='Neighborhood')
Condition1_dummies = pd.get_dummies(ff_house.Condition1, prefix='Condition1')
Condition2_dummies = pd.get_dummies(ff_house.Condition2, prefix='Condition2')
BldgType_dummies = pd.get_dummies(ff_house.BldgType, prefix='BldgType')
HouseStyle_dummies = pd.get_dummies(ff_house.HouseStyle, prefix='HouseStyle')
RoofStyle_dummies = pd.get_dummies(ff_house.RoofStyle, prefix='RoofStyle')
Foundation_dummies = pd.get_dummies(ff_house.Foundation, prefix='Foundation')
BsmtQual_dummies = pd.get_dummies(ff_house.BsmtQual, prefix='BsmtQual')
BsmtExposure_dummies = pd.get_dummies(ff_house.BsmtExposure, prefix='BsmtExposure')
BsmtFinSF2_dummies = pd.get_dummies(ff_house.BsmtFinSF2, prefix='BsmtFinSF2')
CentralAir_dummies = pd.get_dummies(ff_house.CentralAir, prefix='CentralAir')
PavedDrive_dummies = pd.get_dummies(ff_house.PavedDrive, prefix='PavedDrive')
SaleType_dummies = pd.get_dummies(ff_house.SaleType, prefix='SaleType')


In [18]:
dum_ff_house = pd.concat([ff_house, 
                          MSZoning_dummies, 
                          Street_dummies,
                          LotShape_dummies,
                          LandContour_dummies,
                          LotConfig_dummies,
                          LandSlope_dummies,
                          Neighborhood_dummies,
                          Condition1_dummies,
                          Condition2_dummies,
                          BldgType_dummies,
                          HouseStyle_dummies,
                          RoofStyle_dummies,
                          Foundation_dummies,
                          BsmtQual_dummies,
                          BsmtExposure_dummies,
                          BsmtFinSF2_dummies,
                          CentralAir_dummies,
                          PavedDrive_dummies,
                          SaleType_dummies,
                         ], axis=1)

# dum_ff_house = pd.concat([ff_house, Street_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, LotShape_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, LandContour_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, LotConfig_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, LandSlope_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, Neighborhood_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, Condition1_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, Condition2_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, BldgType_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, HouseStyle_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, RoofStyle_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, Foundation_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, BsmtQual_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, BsmtExposure_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, BsmtFinSF2_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, CentralAir_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, PavedDrive_dummies], axis=1)
# dum_ff_house = pd.concat([ff_house, SaleType_dummies], axis=1)


In [19]:
dum_ff_house = dum_ff_house.drop(['MSZoning', 
                    'Street', 
                    'LotShape', 
                    'LandContour', 
                    'LotConfig', 
                    'LandSlope',
                    'Neighborhood',
                    'Condition1',
                    'Condition2',
                    'BldgType',
                    'HouseStyle',
                    'RoofStyle',
                    'Foundation',
                    'BsmtQual',
                    'BsmtExposure',
                    'CentralAir',
                    'GarageType',
                    'GarageYrBlt',
                    'GarageFinish',
                    'PavedDrive',
                    'SaleType',
                    'Bedroom',
                    'Kitchen',
                   ], axis = 1)

In [20]:
dum_ff_house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Columns: 278 entries, MSSubClass to SaleType_WD
dtypes: int64(21), uint8(257)
memory usage: 613.1 KB


In [21]:
dum_ff_house

,MSSubClass,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,60,8450,2003,2003,706,0,150,856,856,854,...,1,0,0,0,0,0,0,0,0,1
1,20,9600,1976,1976,978,0,284,1262,1262,0,...,1,0,0,0,0,0,0,0,0,1
2,60,11250,2001,2002,486,0,434,920,920,866,...,1,0,0,0,0,0,0,0,0,1
3,70,9550,1915,1970,216,0,540,756,961,756,...,1,0,0,0,0,0,0,0,0,1
4,60,14260,2000,2000,655,0,490,1145,1145,1053,...,1,0,0,0,0,0,0,0,0,1
5,50,14115,1993,1995,732,0,64,796,796,566,...,1,0,0,0,0,0,0,0,0,1
6,20,10084,2004,2005,1369,0,317,1686,1694,0,...,1,0,0,0,0,0,0,0,0,1
7,60,10382,1973,1973,859,32,216,1107,1107,983,...,1,0,0,0,0,0,0,0,0,1
8,50,6120,1931,1950,0,0,952,952,1022,752,...,1,0,0,0,0,0,0,0,0,1
9,190,7420,1939,1950,851,0,140,991,1077,0,...,1,0,0,0,0,0,0,0,0,1


In [22]:
# Train a model on pre-2010 data and evaluate its performance on the 2010 houses.

dum_ff_house['YrSold'].value_counts()

2009    336
2007    328
2006    312
2008    302
2010    172
Name: YrSold, dtype: int64

In [23]:
X_train = dum_ff_house.loc[dum_ff_house['YrSold'] < 2010]

In [24]:
X_train['YrSold'].value_counts()

2009    336
2007    328
2006    312
2008    302
Name: YrSold, dtype: int64

In [25]:
X_test = dum_ff_house.loc[dum_ff_house['YrSold'] == 2010]

In [26]:
X_test['YrSold'].value_counts()

2010    172
Name: YrSold, dtype: int64

In [27]:
pre_ten_house = house.loc[house['YrSold'] < 2010]

In [28]:
y_train = pre_ten_house['SalePrice']

In [29]:
y_train.shape

(1278,)

In [30]:
ten_house = house.loc[house['YrSold'] == 2010]

In [31]:
y_test = ten_house['SalePrice']

In [32]:
y_test.shape

(172,)

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, ShuffleSplit


In [34]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_regression, k=40)),
    ('scaler', StandardScaler()),
    ('sfm', SelectFromModel(Lasso())),
    ('regr', ElasticNet())
])

In [35]:
params = {
    'regr__l1_ratio':[.1,.3,.5,.7,.9],
    'regr__alpha':np.logspace(-3,3,7),
    'skb__k':[50, 40],
    'sfm__estimator':[Lasso(), Ridge()]
}

In [36]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                     )

In [37]:
gspipe.fit(X_train, y_train.ravel())

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=40, score_func=<function f_regression at 0x7f7967ad38c8>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sfm', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precom...alse, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'regr__alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03]), 'regr__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], 'skb__k': [50, 40], 'sfm__estimator': [Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 

In [38]:
gspipe.best_score_

0.78804081179539898

In [39]:
gspipe.score(X_test, y_test)

0.85267801892001693

In [40]:
gspipe.best_estimator_

Pipeline(steps=[('skb', SelectKBest(k=50, score_func=<function f_regression at 0x7f7967ad38c8>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sfm', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precom...alse, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))])

In [41]:
skb_mask = gspipe.best_estimator_.named_steps['skb'].get_support()
sfm_mask = gspipe.best_estimator_.named_steps['sfm'].get_support()
X_train.columns[skb_mask][sfm_mask]

Index(['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea',
       'MSZoning_RL', 'MSZoning_RM', 'LotShape_IR1', 'LotShape_Reg',
       'Neighborhood_BrkSide', 'Neighborhood_Edwards', 'Neighborhood_NAmes',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'BldgType_1Fam',
       'HouseStyle_1.5Fin', 'HouseStyle_2Story', 'RoofStyle_Gable',
       'RoofStyle_Hip', 'Foundation_BrkTil', 'Foundation_CBlock',
       'Foundation_PConc', 'BsmtQual_Ex', 'BsmtQual_Fa', 'BsmtQual_Gd',
       'BsmtQual_TA', 'BsmtExposure_Av', 'BsmtExposure_Gd', 'BsmtExposure_NA',
       'BsmtExposure_No', 'CentralAir_N', 'PavedDrive_N', 'PavedDrive_Y',
       'SaleType_New', 'SaleType_WD'],
      dtype='object')

In [42]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)

gspipe_results.sort_values('rank_test_score', ascending=True).T

,72,76,40,68,44,48,52,64,56,20,...,124,125,120,121,131,130,127,126,123,122
mean_fit_time,0.188947,0.180614,0.190607,0.167626,0.184062,0.192348,0.194009,0.178346,0.198885,0.281762,...,0.15775,0.146072,0.162705,0.137107,0.0621372,0.158035,0.114766,0.159014,0.0661117,0.167177
mean_score_time,0.0277297,0.0082016,0.0224615,0.0297463,0.0265871,0.0230777,0.0240018,0.01348,0.0292994,0.0291912,...,0.0174077,0.0184612,0.0327832,0.0324347,0.000893911,0.00111882,0.000892957,0.00090758,0.000864744,0.00100342
mean_test_score,0.788041,0.786486,0.786188,0.785962,0.785477,0.784534,0.783215,0.782289,0.781015,0.780839,...,0.0144445,0.013822,0.0108065,0.0103104,0.00875673,0.00839737,0.00559858,0.00534136,0.00383508,0.0036354
mean_train_score,0.825557,0.833565,0.833982,0.817913,0.834806,0.835616,0.836384,0.81043,0.837107,0.837149,...,0.0170945,0.0164309,0.0134325,0.0129038,0.0114485,0.0109857,0.00824713,0.00791514,0.0064596,0.00620121
param_regr__alpha,1,1,0.1,1,0.1,0.1,0.1,1,0.1,0.01,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
param_regr__l1_ratio,0.7,0.9,0.1,0.5,0.3,0.5,0.7,0.3,0.9,0.1,...,0.3,0.3,0.1,0.1,0.5,0.5,0.3,0.3,0.1,0.1
param_sfm__estimator,"Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...",...,"Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Lasso(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...","Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr..."
param_skb__k,50,50,50,50,50,50,50,50,50,50,...,50,40,50,40,40,50,40,50,40,50
params,"{'regr__alpha': 1.0, 'regr__l1_ratio': 0.7, 's...","{'regr__alpha': 1.0, 'regr__l1_ratio': 0.9, 's...","{'regr__alpha': 0.1, 'regr__l1_ratio': 0.1, 's...","{'regr__alpha': 1.0, 'regr__l1_ratio': 0.5, 's...","{'regr__alpha': 0.1, 'regr__l1_ratio': 0.3, 's...","{'regr__alpha': 0.1, 'regr__l1_ratio': 0.5, 's...","{'regr__alpha': 0.1, 'regr__l1_ratio': 0.7, 's...","{'regr__alpha': 1.0, 'regr__l1_ratio': 0.3, 's...","{'regr__alpha': 0.1, 'regr__l1_ratio': 0.9, 's...","{'regr__alpha': 0.01, 'regr__l1_ratio': 0.1, '...",...,"{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.3,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.3,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.1,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.1,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.5,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.5,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.3,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.3,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.1,...","{'regr__alpha': 1000.0, 'regr__l1_ratio': 0.1,..."
rank_test_score,1,2,3,4,5,6,7,8,9,10,...,131,132,133,134,135,136,137,138,139,140


In [43]:
gspipe.best_estimator_.named_steps['sfm'].get_support

<bound method SelectorMixin.get_support of SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
        prefit=False, threshold=None)>

In [44]:
# Characterize your model. How well does it perform? What are the best estimates of price?

#1. Model performance - the Lasso model performed significanly better in the training set - 0.94 - 
#        compared to the test set - 0.85 - this is not surprising that the score dropped on the test set, however 
#        I am suprised that the training set dropped significantly, almost 10 points! The model is possible overfit.
#        This would explain the dramatic drop in performance. Perhaps re-selecting the fixed feature might yield 
#        better results, or trying additional regressor models.

#2. Best estimates of price - According to the above table (see Out[215]) Lasso with a param_regr__l1_ratio of 0.9,
#        meaning that Lassor was selected for the majority of the model, however, I would have thought Lasso
#        would have knocked out some feature, but I looks like the model performed better with 50 feature selected.


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [45]:
#Getting test residuals

y_reno_test = gspipe.predict(X_test) - y_test 

In [46]:
#Getting absolute values for test residuals

abs_y_reno_test = abs(y_reno_test)

In [47]:
#Getting train residuals

y_reno_train = gspipe.predict(X_train) - y_train

In [48]:
#Getting absolute values for train residuals

abs_y_reno_train = abs(y_reno_train)

In [49]:
abs_y_reno_train

0        2342.669686
1        9275.211247
2        7488.582617
3       34524.944677
4       61068.851534
5       31019.067454
6       29920.078417
7       16145.228674
8       21461.921209
9        8935.005306
10       9148.355456
11      12597.921897
12      14374.204269
13      50355.076206
14       5976.180817
15       6831.068049
17      32878.372840
18      13129.999959
19        150.347458
20      25730.727792
21      37622.086017
22      14469.925629
23       3971.364955
25       6269.744630
28      12419.063980
29       3762.512186
31        586.638874
32      42963.241647
34       5495.407201
35      25731.265912
            ...     
1424     5985.122223
1425     8331.339014
1426     5832.916761
1427      638.115379
1429     2114.417898
1430    17719.040295
1431    12163.035436
1432    57633.802471
1433    21167.100434
1434     5890.635245
1435      915.189583
1436     5091.981471
1437    48617.289463
1439    15600.688501
1440    61577.764648
1441    18712.671961
1442    11376

In [50]:
#Getting the data set of renovatable features

reno_house = house.loc[:, ['OverallQual',
                            'OverallCond',
                            'RoofMatl',
                            'Exterior1st',
                            'Exterior2nd',
                            'MasVnrType',
                            'MasVnrArea',
                            'ExterQual',
                            'ExterCond',
                            'BsmtCond',
                            'BsmtFinType1',
                            'BsmtFinType2',
                            'Heating',
                            'HeatingQC',
                            'Electrical',
                            'LowQualFinSF',
                            'KitchenQual',
                            'Functional',
                            'GarageQual',
                            'GarageCond',
                            'WoodDeckSF',
                            'OpenPorchSF',
                            'EnclosedPorch',
                            '3SsnPorch',
                            'ScreenPorch',
                            'MiscVal',
                            'SaleCondition',
                            'YrSold'
                          ]]

In [51]:
reno_house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Data columns (total 28 columns):
OverallQual      1450 non-null int64
OverallCond      1450 non-null int64
RoofMatl         1450 non-null object
Exterior1st      1450 non-null object
Exterior2nd      1450 non-null object
MasVnrType       1450 non-null object
MasVnrArea       1450 non-null float64
ExterQual        1450 non-null object
ExterCond        1450 non-null object
BsmtCond         1450 non-null object
BsmtFinType1     1450 non-null object
BsmtFinType2     1450 non-null object
Heating          1450 non-null object
HeatingQC        1450 non-null object
Electrical       1450 non-null object
LowQualFinSF     1450 non-null int64
KitchenQual      1450 non-null object
Functional       1450 non-null object
GarageQual       1450 non-null object
GarageCond       1450 non-null object
WoodDeckSF       1450 non-null int64
OpenPorchSF      1450 non-null int64
EnclosedPorch    1450 non-null int64
3SsnPorch        1450 no

In [52]:
# Dummying the renovatable data set. Just had a major break through! I just learned you can pass the entire dataframe 
# into get_dummies and it will recognize continuous values and keep them continuouse!

reno_dummies = pd.get_dummies(reno_house)

In [53]:
reno_dummies

,OverallQual,OverallCond,MasVnrArea,LowQualFinSF,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,...,GarageCond_Gd,GarageCond_NA,GarageCond_Po,GarageCond_TA,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,7,5,196.0,0,0,61,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,6,8,0.0,0,298,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,7,5,162.0,0,0,42,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,7,5,0.0,0,0,35,272,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,8,5,350.0,0,192,84,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5,5,5,0.0,0,40,30,0,320,0,700,...,0,0,0,1,0,0,0,0,1,0
6,8,5,186.0,0,255,57,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
7,7,6,240.0,0,235,204,228,0,0,350,...,0,0,0,1,0,0,0,0,1,0
8,7,5,0.0,0,90,0,205,0,0,0,...,0,0,0,1,1,0,0,0,0,0
9,5,6,0.0,0,0,4,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [54]:
# Filtering traint set for years < 2010

reno_X_train = reno_dummies[reno_dummies['YrSold'] < 2010]

In [55]:
reno_X_train.shape

(1278, 129)

In [56]:
# Filtering test set for years = 2010

reno_X_test = reno_dummies[reno_dummies['YrSold'] == 2010]

In [57]:
reno_X_test.shape

(172, 129)

In [58]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_regression, k=40)),
    ('scaler', StandardScaler()),
    ('sfm', SelectFromModel(Lasso())),
    ('regr', ElasticNet())
])

In [59]:
params = {
    'regr__l1_ratio':[.1,.3,.5,.7,.9],
    'regr__alpha':np.logspace(-3,3,7),
    'skb__k':[50, 40],
    'sfm__estimator':[Lasso(), Ridge()]
}

In [60]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                     )

In [61]:
gspipe.fit(reno_X_train, abs_y_reno_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=40, score_func=<function f_regression at 0x7f7967ad38c8>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sfm', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precom...alse, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'regr__alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03]), 'regr__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], 'skb__k': [50, 40], 'sfm__estimator': [Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 

In [62]:
gspipe.score(reno_X_train, abs_y_reno_train)

0.36367677513864283

In [63]:
gspipe.fit(reno_X_test, abs_y_reno_test)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=40, score_func=<function f_regression at 0x7f7967ad38c8>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sfm', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precom...alse, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'regr__alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03]), 'regr__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], 'skb__k': [50, 40], 'sfm__estimator': [Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 

In [64]:
gspipe.score(reno_X_test, abs_y_reno_test)

0.34289635003014962

In [65]:
skb_mask = gspipe.best_estimator_.named_steps['skb'].get_support()
sfm_mask = gspipe.best_estimator_.named_steps['sfm'].get_support()
reno_X_test.columns[skb_mask][sfm_mask]

Index(['OverallQual', 'MasVnrArea', 'OpenPorchSF', 'ScreenPorch', 'MiscVal',
       'Exterior1st_BrkFace', 'Exterior1st_CemntBd', 'Exterior1st_Stucco',
       'Exterior2nd_AsphShn', 'Exterior2nd_Wd Shng', 'MasVnrType_None',
       'MasVnrType_Stone', 'ExterQual_Ex', 'ExterQual_TA', 'ExterCond_Po',
       'BsmtFinType1_ALQ', 'BsmtFinType1_BLQ', 'BsmtFinType1_GLQ',
       'BsmtFinType1_LwQ', 'BsmtFinType1_Unf', 'BsmtFinType2_ALQ',
       'BsmtFinType2_GLQ', 'BsmtFinType2_Rec', 'BsmtFinType2_Unf',
       'HeatingQC_Ex', 'HeatingQC_Fa', 'HeatingQC_Gd', 'Electrical_FuseA',
       'Electrical_SBrkr', 'KitchenQual_Ex', 'KitchenQual_Gd',
       'KitchenQual_TA', 'Functional_Mod', 'Functional_Typ', 'GarageQual_NA',
       'GarageQual_TA', 'GarageCond_NA', 'GarageCond_TA',
       'SaleCondition_Normal', 'SaleCondition_Partial'],
      dtype='object')

In [66]:
import numpy as np

In [67]:
# 1. Evaluate the effect in dollars of the renovate-able features.
#         A: (this is a long shot) So, if 25% is the explained variance of the renovatable model, then took the
# .          the average of the residuals, then multiplied it by the explained variance, you could argue that
# .          renovations could account for ~$5,000 (see cell below) of the sales price.

average_residual = np.average(abs_y_reno_test)

average_residual

explained_reno_variance = 0.25363679806781125

effect_in_dollars = average_residual * explained_reno_variance

In [68]:
effect_in_dollars

4951.5574419556297

In [69]:
# 2a. How would your company use this second model and its coefficients to determine whether they should buy a property
# .   or not? 

# .      A: The company would use the second model to determine which house to select given a similar set 
# .         fixed features. For example, if two houses were equal in price, but one had renovatable features like
# .         an Open Porch, the company should select the one with renovatable features becuase it can be used to 
# .         increase the sales price. 
# .         
# .         Also, If the company is only looking to renovate, then they should select houses with the top renovatable
# .         features (see Out[360]), or houses without the features and then added them.
                  
       
# 2b. Explain how the company can use the two models you have built to determine if they can make money.

# .      A: The company can . . . 
# .         A1: can purchase houses with fixed features that have been under valued.
# .         A2: add renovatable feature to exiting houses.
# .         A3: renovate renovatable features. 

In [70]:
# 3. Investigate how much of the variance in price remaining is explained by these features.

# .      A: The fixed features model explains doesn't explain 15% of the variance. Of that 15% the renovatable model
# .         explains 25% of the model. 25% of 15% is 3.75%. Therefore, the renovatable model explains 3.75% of the
# .         variance in price of the fixed model.

In [71]:
# 4. Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

# .      A: Yes, but it seems that the dollar amount associate (if correct) with the sale of a house from renovatable
# .         feature is too low. I'm not sure what the over head is on running the company, but purchasing a house and
# .         renovating it for ~5K doesn't seem like enough. 

# .         I would recommend that the company use the first model to buy properties that have been under valued,
# .         and only use the second model if they see value in expending the resources for the potential of a return
# .         of ~5K.